In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import argparse
import os
import tqdm
import scipy.ndimage as nd
#from utils import deprocess, preprocess, clip
from torchvision import transforms

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

preprocess = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])


def deprocess(image_np):
    image_np = image_np.squeeze().transpose(1, 2, 0)
    image_np = image_np * std.reshape((1, 1, 3)) + mean.reshape((1, 1, 3))
    image_np = np.clip(image_np, 0.0, 255.0)
    return image_np


def clip(image_tensor):
    for c in range(3):
        m, s = mean[c], std[c]
        image_tensor[0, c] = torch.clamp(image_tensor[0, c], -m / s, (1 - m) / s)
    return image_tensor

def norm(x):
    return np.array((x - np.min(x)) / (np.max(x) - np.min(x)))

def dream(image, model, iterations, lr):
    """ Updates the image to maximize outputs for n iterations """
    Tensor = torch.cuda.FloatTensor if torch.cuda.is_available else torch.FloatTensor
    image = Variable(Tensor(image), requires_grad=True)
    for i in range(iterations):
        model.zero_grad()
        out = model(image)
        loss = out.norm()
        loss.backward()
        avg_grad = np.abs(image.grad.data.cpu().numpy()).mean()
        norm_lr = lr / avg_grad
        image.data += norm_lr * image.grad.data
        image.data = clip(image.data)
        image.grad.data.zero_()
    return image.cpu().data.numpy()


def deep_dream(image, model, iterations, lr, octave_scale, num_octaves):
    """ Main deep dream method """
    image = preprocess(image).unsqueeze(0).cpu().data.numpy()

    # Extract image representations for each octave
    octaves = [image]
    for _ in range(num_octaves - 1):
        octaves.append(nd.zoom(octaves[-1], (1, 1, 1 / octave_scale, 1 / octave_scale), order=1))

    detail = np.zeros_like(octaves[-1])
    for octave, octave_base in enumerate(tqdm.tqdm(octaves[::-1], desc="Dreaming")):
        if octave > 0:
            # Upsample detail to new octave dimension
            detail = nd.zoom(detail, np.array(octave_base.shape) / np.array(detail.shape), order=1)
        # Add deep dream detail from previous octave to new base
        input_image = octave_base + detail
        # Get new deep dream image
        dreamed_image = dream(input_image, model, iterations, lr)
        # Extract deep dream details
        detail = dreamed_image - octave_base

    return deprocess(dreamed_image)

'''   if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--input_image", type=str, default="images/supermarket.jpg", help="path to input image")
    parser.add_argument("--iterations", default=20, help="number of gradient ascent steps per octave")
    parser.add_argument("--at_layer", default=27, type=int, help="layer at which we modify image to maximize outputs")
    parser.add_argument("--lr", default=0.01, help="learning rate")
    parser.add_argument("--octave_scale", default=1.4, help="image scale between octaves")
    parser.add_argument("--num_octaves", default=10, help="number of octaves")
    args = parser.parse_args()
'''


'   if __name__ == "__main__":\n    parser = argparse.ArgumentParser()\n    parser.add_argument("--input_image", type=str, default="images/supermarket.jpg", help="path to input image")\n    parser.add_argument("--iterations", default=20, help="number of gradient ascent steps per octave")\n    parser.add_argument("--at_layer", default=27, type=int, help="layer at which we modify image to maximize outputs")\n    parser.add_argument("--lr", default=0.01, help="learning rate")\n    parser.add_argument("--octave_scale", default=1.4, help="image scale between octaves")\n    parser.add_argument("--num_octaves", default=10, help="number of octaves")\n    args = parser.parse_args()\n'

In [12]:
#Parametreler
file_type = ".jpg"
input_actual_name =  "baykus5"
input_name =  "\\" +input_actual_name+ file_type
input_image = "E:\\deep dream\\PyTorch-Deep-Dream-master\\PyTorch-Deep-Dream-master\\images" + input_name
image = Image.open(input_image).convert('RGB')
at_layer = 27
iterations = 25
lr = 0.01
octave_scale = 1.4
num_octaves = 10
base_path = "E:\\deep dream\\PyTorch-Deep-Dream-master\\PyTorch-Deep-Dream-master"
output_path = "\\outputs"
Output= "Output_"
OUTPUT_NAME = "\\" +Output +input_actual_name+ file_type
total_path = base_path + output_path +OUTPUT_NAME

# Define the model
network = models.vgg19(pretrained=True)
layers = list(network.features.children())
model = nn.Sequential(*layers[: (at_layer + 1)])

if torch.cuda.is_available:
    model = model.cuda()
print(network)


# Extract deep dream image
dreamed_image = deep_dream(
    image,
    model,
    iterations=iterations,
    lr=lr,
    octave_scale=octave_scale,
    num_octaves=num_octaves,
    )


# Save and plot image
os.makedirs("outputs", exist_ok=True)
filename = input_image.split("//")[-1]
plt.figure(figsize=(20, 20))
plt.imshow(dreamed_image)


plt.imsave(base_path + output_path +OUTPUT_NAME, norm(dreamed_image))
#plt.imsave(f"{base_path}outputs\\output_", norm(dreamed_image))
#plt.show()

Dreaming:   0%|                                                                                 | 0/10 [00:00<?, ?it/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

RuntimeError: Given input size: (512x1x2). Calculated output size: (512x0x1). Output size is too small

In [ ]:
# RuntimeError: Given input size: (512x1x1). Calculated output size: (512x0x0). Output size is too small